In [25]:
!pip install firebase_admin

  Using cached firebase_admin-4.4.0-py3-none-any.whl (111 kB)
  Using cached google_cloud_firestore-1.9.0-py2.py3-none-any.whl (344 kB)
  Using cached CacheControl-0.12.6-py2.py3-none-any.whl (19 kB)
  Using cached google_cloud_storage-1.31.2-py2.py3-none-any.whl (88 kB)
  Using cached google_api_python_client-1.12.3-py2.py3-none-any.whl (61 kB)
  Using cached google_api_core-1.22.4-py2.py3-none-any.whl (91 kB)
  Using cached google_cloud_core-1.4.3-py2.py3-none-any.whl (27 kB)
     |████████████████████████████████| 274 kB 5.2 MB/s eta 0:00:01
  Using cached google_resumable_media-1.1.0-py2.py3-none-any.whl (75 kB)
  Using cached google_auth_httplib2-0.0.4-py2.py3-none-any.whl (9.1 kB)
  Using cached uritemplate-3.0.1-py2.py3-none-any.whl (15 kB)
  Using cached httplib2-0.18.1-py3-none-any.whl (95 kB)
     |████████████████████████████████| 1.3 MB 3.7 MB/s eta 0:00:01
  Using cached googleapis_common_protos-1.52.0-py2.py3-none-any.whl (100 kB)
     |████████████████████████████████| 3

In [1]:

import firebase_admin
from firebase_admin import credentials
from firebase_admin import db
from firebase_admin import auth
from firebase_admin import firestore

In [3]:
cred = credentials.Certificate("../site-hopper-adminsdk.json")
firebase_admin.initialize_app(cred)


In [4]:
db = firestore.client()

In [29]:
users_ref = db.collection(u'users')
docs = users_ref.stream()

for doc in docs:
    print(f'{doc.id} => {doc.to_dict()}')
  

uid1 => {'name': 'Tushar', 'list_comment': [<google.cloud.firestore_v1.document.DocumentReference object at 0x7f3ab090cc50>, <google.cloud.firestore_v1.document.DocumentReference object at 0x7f3ab090c9e8>, <google.cloud.firestore_v1.document.DocumentReference object at 0x7f3ad3879ac8>]}
uid2 => {'list_comment': [<google.cloud.firestore_v1.document.DocumentReference object at 0x7f3ab090ccf8>, <google.cloud.firestore_v1.document.DocumentReference object at 0x7f3ab090cbe0>, <google.cloud.firestore_v1.document.DocumentReference object at 0x7f3ab090c978>], 'name': 'Akshay'}
uid3 => {'list_comment': [<google.cloud.firestore_v1.document.DocumentReference object at 0x7f3ab090c9e8>, <google.cloud.firestore_v1.document.DocumentReference object at 0x7f3ab090ca90>], 'name': 'ellon'}


In [5]:
# Note: Use of CollectionRef stream() is prefered to get()
docs = db.collection(u'domains').stream()

for doc in docs:
    print(f'{doc.id} => {doc.to_dict()}')

stackoverflow.com => {}
youtube.com => {}


### Get all comments

In [35]:
query = db.collection('domains').document('stackoverflow.com').collection("routes").\
    document("question::1").collection("comments")
docs = query.stream()
for doc in docs:
    comment = doc.to_dict()
    print(f'{doc.id} => {comment["body"]} rating {comment["rating"]} created on: {comment.get("created_on","")}')
    if len(comment.get("list_reply",[]))>0:
        list_reply = comment.get('list_reply')
        print(f"\t replies {len(list_reply)}")
        for reply in list_reply:
            comment = reply.get().to_dict()
            print(f'\t {reply.id} => {comment["body"]} created on: {comment.get("created_on","")}')
    

uid1 => what is this rating 3 created on: 2020-10-10 15:30:00+00:00
	 replies 2
	 uid2 => I also dont know created on: 2020-10-10 15:40:00+00:00
	 uid3 => lets go in space created on: 2020-10-10 15:41:00+00:00
uid2 => I also dont know rating 1 created on: 2020-10-10 15:40:00+00:00
uid3 => lets go in space rating 5 created on: 2020-10-10 15:41:00+00:00


In [45]:
query = db.collection('domains').document('stackoverflow.com').collection("routes").\
    document("question::2").collection("comments").where(u"parent",u"==","")
docs = query.stream()
for doc in docs:
    comment = doc.to_dict()
    print(f'{doc.id} => {comment["body"]} created on: {comment.get("created_on","")}')
    if len(comment.get("list_reply",[]))>0:
        list_reply = comment.get('list_reply')
        print(f"\t replies {len(list_reply)}")
        for reply in list_reply:
            comment = reply.get().to_dict()
            print(f'\t {reply.id} => {comment["body"]} created on: {comment.get("created_on","")}')
    

uid4 => something is fishy created on: 
	 replies 1
	 uid5 => you are dumb created on: 


In [57]:
query = db.collection('domains').document('facebook.com').collection("routes").\
    document("tushr").collection("comments")
docs = query.stream()
for doc in docs:
    comment = doc.to_dict()
    print(f'{doc.id} => {comment["body"]} created on: {comment.get("created_on","")}')
    print(comment)
    reply_collection = query.document(doc.id).collection("replies").get()
    if len(reply_collection)>0:
        for reply in reply_collection:
            print(reply.to_dict())

158e78489c504940b410ba92fa7982e5 => just trying created on: 2020-10-11 13:15:45.396348+00:00
{'parent': None, 'body': 'just trying', 'rating': 0, 'list_reply': None, 'created_on': DatetimeWithNanoseconds(2020, 10, 11, 13, 15, 45, 396348, tzinfo=<UTC>), 'user': {'id': 'uid1', 'name': 'Tushar'}}
{'commenter': {'name': 'Akshay', 'id': 'uid2'}, 'rating': 0, 'domain': 'facebook.com', 'body': 'reply 1', 'list_parent': ['158e78489c504940b410ba92fa7982e5'], 'route': 'tushr'}
d054518a8f1a466d86f107d0204994f9 => new comment created on: 2020-10-11 14:09:31.921230+00:00
{'user': {'name': 'Akshay', 'id': 'uid2'}, 'list_reply': None, 'created_on': DatetimeWithNanoseconds(2020, 10, 11, 14, 9, 31, 921230, tzinfo=<UTC>), 'body': 'new comment', 'parent': None, 'rating': 0}


### Get top comments for specific post in specific domain order by rating 


In [6]:
query = db.collection('domains').document('stackoverflow.com').collection("routes").document("question::1").collection("comments").order_by("rating").limit(3)
docs = query.stream()
for doc in docs:
    print(f'{doc.id} => {doc.to_dict()}')

uid2 => {'created_on': DatetimeWithNanoseconds(2020, 10, 10, 15, 40, tzinfo=<UTC>), 'parent': <google.cloud.firestore_v1.document.DocumentReference object at 0x000001E627A2B608>, 'rating': 1, 'body': 'I also dont know', 'user': <google.cloud.firestore_v1.document.DocumentReference object at 0x000001E627A2BE88>}
uid1 => {'created_on': DatetimeWithNanoseconds(2020, 10, 10, 15, 30, tzinfo=<UTC>), 'list_reply': [<google.cloud.firestore_v1.document.DocumentReference object at 0x000001E62985E608>, <google.cloud.firestore_v1.document.DocumentReference object at 0x000001E62985E908>], 'rating': 3, 'body': 'what is this', 'user': <google.cloud.firestore_v1.document.DocumentReference object at 0x000001E62985E248>}
uid3 => {'created_on': DatetimeWithNanoseconds(2020, 10, 10, 15, 41, tzinfo=<UTC>), 'parent': <google.cloud.firestore_v1.document.DocumentReference object at 0x000001E62985CFC8>, 'rating': 5, 'body': 'lets go in space', 'user': <google.cloud.firestore_v1.document.DocumentReference objec

### Get top reply for specific comment order by time

In [7]:
doc = db.collection('domains').document('stackoverflow.com')\
.collection("routes").document("question::1").collection("comments").document("uid1")

comment_dict = doc.get().to_dict()
list_reply = comment_dict["list_reply"]
for reply in list_reply[:1]:
    print(reply.get().to_dict()["body"])


I also dont know


### Get top comments and reply by specific user for all posts in all domain.

In [32]:
doc = db.collection('users').document('uid2')

user_dict = doc.get().to_dict()
list_comment = user_dict["list_comment"]

for comment in list_comment[:3]:
    parent = comment.get().to_dict().get("parent",None)
    if parent is not None:
        print("parent : "+parent.get().to_dict()["body"])
        
    
    print("Body : "+comment.get().to_dict()["body"])
    print(comment.get().to_dict())

parent : what is this
Body : I also dont know
{'created_on': DatetimeWithNanoseconds(2020, 10, 10, 15, 40, tzinfo=<UTC>), 'parent': <google.cloud.firestore_v1.document.DocumentReference object at 0x000001E62985E348>, 'rating': 1, 'body': 'I also dont know', 'user': <google.cloud.firestore_v1.document.DocumentReference object at 0x000001E62985E188>}
parent : something is fishy
Body : you are dumb
{'parent': <google.cloud.firestore_v1.document.DocumentReference object at 0x000001E6298D5A88>, 'rating': 5, 'body': 'you are dumb', 'user': <google.cloud.firestore_v1.document.DocumentReference object at 0x000001E6298D5D08>}
parent : this is awesome
Body : yes it is
{'parent': <google.cloud.firestore_v1.document.DocumentReference object at 0x000001E6298D5C08>, 'rating': 4, 'body': 'yes it is', 'user': <google.cloud.firestore_v1.document.DocumentReference object at 0x000001E6298D5B48>}


3

In [40]:
doc = db.collection("domain").document("stackoverflow.com/routes/question::2/comments/uuid4")

In [42]:
print(doc.get().to_dict())


None


In [43]:
doc

In [1]:
import uuid

In [2]:
i = uuid.uuid4()

In [6]:
i

UUID('f2c2cd51-bb94-45d1-93e6-d75f5c783e89')